# Lab 2.4.1 Solutions: Hub Exploration

This notebook contains solutions to the exercises in the Hub Exploration notebook.

In [ ]:
# Setup
from huggingface_hub import HfApi, list_models
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
import json
from datetime import datetime

api = HfApi()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Exercise: Document 10 Models

Complete solution for documenting 10 models from different task categories.

In [ ]:
def document_model(model_id: str) -> dict:
    """Create documentation for a Hugging Face model."""
    try:
        info = api.model_info(model_id)
        return {
            "model_id": model_id,
            "author": info.author,
            "task": info.pipeline_tag,
            "downloads": info.downloads,
            "likes": info.likes,
            "library": info.library_name,
            "tags": info.tags[:10] if info.tags else [],
            "documented_at": datetime.now().isoformat(),
            "notes": "",
            "tested_locally": False
        }
    except Exception as e:
        return {"model_id": model_id, "error": str(e)}

# List of 10 models across different tasks
models_to_document = [
    # Text Classification (2 models)
    "distilbert-base-uncased-finetuned-sst-2-english",
    "cardiffnlp/twitter-roberta-base-sentiment-latest",
    
    # Text Generation (2 models)
    "gpt2",
    "distilgpt2",
    
    # Question Answering (2 models)
    "distilbert-base-cased-distilled-squad",
    "deepset/roberta-base-squad2",
    
    # Named Entity Recognition (2 models)
    "dslim/bert-base-NER",
    "Jean-Baptiste/camembert-ner",
    
    # Other (2 models - summarization and translation)
    "facebook/bart-large-cnn",
    "Helsinki-NLP/opus-mt-en-de"
]

# Document all models
my_model_docs = []
print("Documenting models...\n")

for model_id in models_to_document:
    doc = document_model(model_id)
    my_model_docs.append(doc)
    if "error" not in doc:
        print(f"[OK] {model_id}")
        print(f"     Task: {doc['task']}, Downloads: {doc['downloads']:,}")
    else:
        print(f"[ERR] {model_id}: {doc['error']}")

print(f"\nTotal models documented: {len(my_model_docs)}")

In [ ]:
# Display documentation summary
print("\nModel Documentation Summary:")
print("=" * 80)
print(f"{'Model ID':<50} {'Task':<20} {'Downloads':>10}")
print("-" * 80)

for doc in my_model_docs:
    if "error" not in doc:
        print(f"{doc['model_id']:<50} {doc['task'] or 'N/A':<20} {doc['downloads']:>10,}")

## Exercise: Find a Hidden Gem

Finding an underrated model with fewer than 10,000 downloads.

In [ ]:
# Search for less popular but potentially useful models
# Let's look for specialized sentiment models

# Search for financial sentiment models (specialized domain)
financial_models = list(api.list_models(
    search="financial sentiment",
    sort="downloads",
    direction=1,  # Ascending - fewer downloads first
    limit=20
))

print("Potential Hidden Gems (Financial Sentiment):")
print("=" * 60)

for m in financial_models:
    downloads = m.downloads if hasattr(m, 'downloads') else 0
    if 100 < downloads < 10000:  # Has some usage but not too popular
        print(f"\n{m.id}")
        print(f"  Downloads: {downloads:,}")
        print(f"  Task: {m.pipeline_tag if hasattr(m, 'pipeline_tag') else 'N/A'}")

In [ ]:
# My Hidden Gem Selection and Analysis
hidden_gem = "nickmuchi/finbert-tone-finetuned-finance-topic-classification"

try:
    gem_doc = document_model(hidden_gem)
    
    print("\nMY HIDDEN GEM ANALYSIS")
    print("=" * 60)
    print(f"Model: {hidden_gem}")
    print(f"Author: {gem_doc.get('author', 'Unknown')}")
    print(f"Task: {gem_doc.get('task', 'Unknown')}")
    print(f"Downloads: {gem_doc.get('downloads', 0):,}")
    print(f"\nWhy I think this is underrated:")
    print("  - Specialized for financial text classification")
    print("  - Built on FinBERT which is domain-adapted")
    print("  - Useful for fintech applications")
    print("  - Has clear documentation")
except Exception as e:
    print(f"Could not analyze hidden gem: {e}")
    print("\nAlternative: Try 'yiyanghkust/finbert-tone' for financial sentiment")

## Exercise: Test 3 Models Locally

Complete solution for testing models locally with detailed results.

In [ ]:
import time
from transformers import (
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    AutoModelForCausalLM
)

def test_model_locally(model_id: str, task: str, test_input: str, context: str = None):
    """Test a model locally and return results."""
    results = {
        "model_id": model_id,
        "task": task,
        "success": False,
        "output": None,
        "load_time": 0,
        "inference_time_ms": 0,
        "memory_gb": 0,
        "error": None
    }
    
    try:
        torch.cuda.empty_cache()
        initial_mem = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
        
        # Load
        start = time.time()
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        
        if task == "classification":
            model = AutoModelForSequenceClassification.from_pretrained(
                model_id, torch_dtype=torch.bfloat16
            ).to(device).eval()
        elif task == "generation":
            model = AutoModelForCausalLM.from_pretrained(
                model_id, torch_dtype=torch.bfloat16
            ).to(device).eval()
            tokenizer.pad_token = tokenizer.eos_token
        elif task == "qa":
            model = AutoModelForQuestionAnswering.from_pretrained(
                model_id, torch_dtype=torch.bfloat16
            ).to(device).eval()
        
        results["load_time"] = time.time() - start
        results["memory_gb"] = (torch.cuda.memory_allocated() / 1e9) - initial_mem
        
        # Inference
        start = time.time()
        
        if task == "qa" and context:
            inputs = tokenizer(test_input, context, return_tensors="pt", truncation=True)
        else:
            inputs = tokenizer(test_input, return_tensors="pt", truncation=True)
        
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            if task == "classification":
                outputs = model(**inputs)
                probs = torch.softmax(outputs.logits, dim=1)
                pred = torch.argmax(probs, dim=1).item()
                conf = probs[0][pred].item()
                label = model.config.id2label.get(pred, f"Class {pred}")
                results["output"] = f"{label} ({conf:.2%})"
            elif task == "generation":
                outputs = model.generate(**inputs, max_new_tokens=30, do_sample=True, temperature=0.7)
                results["output"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
            elif task == "qa":
                outputs = model(**inputs)
                start_idx = torch.argmax(outputs.start_logits)
                end_idx = torch.argmax(outputs.end_logits) + 1
                results["output"] = tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx])
        
        results["inference_time_ms"] = (time.time() - start) * 1000
        results["success"] = True
        
        del model, tokenizer
        torch.cuda.empty_cache()
        
    except Exception as e:
        results["error"] = str(e)
    
    return results

In [ ]:
# Test 3 models
tests = [
    {
        "model_id": "distilbert-base-uncased-finetuned-sst-2-english",
        "task": "classification",
        "test_input": "This product exceeded all my expectations! Best purchase ever."
    },
    {
        "model_id": "gpt2",
        "task": "generation",
        "test_input": "The future of artificial intelligence is"
    },
    {
        "model_id": "distilbert-base-cased-distilled-squad",
        "task": "qa",
        "test_input": "What is the capital of France?",
        "context": "France is a country in Western Europe. Its capital is Paris, a beautiful city known for the Eiffel Tower."
    }
]

print("LOCAL MODEL TESTING RESULTS")
print("=" * 80)

for test in tests:
    print(f"\nTesting: {test['model_id']}")
    print(f"Task: {test['task']}")
    print(f"Input: {test['test_input'][:50]}...")
    
    result = test_model_locally(
        test['model_id'],
        test['task'],
        test['test_input'],
        test.get('context')
    )
    
    if result["success"]:
        print(f"\nStatus: SUCCESS")
        print(f"Output: {result['output']}")
        print(f"Load time: {result['load_time']:.2f}s")
        print(f"Inference: {result['inference_time_ms']:.2f}ms")
        print(f"Memory: {result['memory_gb']:.2f} GB")
    else:
        print(f"\nStatus: FAILED")
        print(f"Error: {result['error']}")
    
    print("-" * 60)

### Expected Output (Sample)

When you run the tests above, you should see output similar to:

```
LOCAL MODEL TESTING RESULTS
================================================================================

Testing: distilbert-base-uncased-finetuned-sst-2-english
Task: classification
Input: This product exceeded all my expectations! Best pu...

Status: SUCCESS
Output: POSITIVE (99.87%)
Load time: 1.23s
Inference: 12.45ms
Memory: 0.26 GB
------------------------------------------------------------

Testing: gpt2
Task: generation
Input: The future of artificial intelligence is...

Status: SUCCESS
Output: The future of artificial intelligence is uncertain, but one thing is certain: it will change the world in ways we cannot yet imagine.
Load time: 2.15s
Inference: 245.32ms
Memory: 0.48 GB
------------------------------------------------------------

Testing: distilbert-base-cased-distilled-squad
Task: qa
Input: What is the capital of France?...

Status: SUCCESS
Output: Paris
Load time: 1.18s
Inference: 8.21ms
Memory: 0.26 GB
------------------------------------------------------------
```

**Note:** Actual values will vary slightly based on your hardware and model versions.

## Summary

In this solution notebook, we:

1. **Documented 10 models** across different task categories:
   - Text Classification (2)
   - Text Generation (2)
   - Question Answering (2)
   - Named Entity Recognition (2)
   - Other tasks (2)

2. **Found a hidden gem**: Specialized financial sentiment model with good documentation but fewer downloads

3. **Tested 3 models locally**:
   - Sentiment classification
   - Text generation
   - Question answering

Key learnings:
- Use `HfApi` for programmatic Hub access
- Model cards are essential for understanding model capabilities
- Different Auto classes for different tasks
- Memory management is important when testing multiple models